In [1]:
Threads.nthreads()

1

In [2]:
using Gridap
import Gridap: ∇
using LinearAlgebra
using GridapGmsh

In [3]:
#Mallado
model = GmshDiscreteModel("test-gridap/circle.msh")
const global degree = 1
Ω = Triangulation(model)
dΩ = Measure(Ω,2*degree)
quad = CellQuadrature(Ω,degree)

Info    : Reading 'test-gridap/circle.msh'...
Info    : 423 nodes
Info    : 844 elements
Info    : Done reading 'test-gridap/circle.msh'


CellQuadrature()

In [4]:
#Parámetros

u_input(x) = 0.  #valor en la frontera izquierda
betaf(x) = VectorValue(-x[2],x[1]) #vector de transporte
u_init(x) = exp(-1000*((x[1]-0.3)^2+(x[2])^2)) #Condición inicial

#Partición en tiempo
const global t_init = 0.
t_end = 2*π

N_intervalos = 100;

n_iter = 1;
T = LinRange(t_init, t_end, N_intervalos+1);

In [5]:
#Espacio de funciones
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe,dirichlet_tags=["Dirichlet"])

U = TrialFESpace(V0)

#beta = interpolate_everywhere(betaf,U)
#beta = get_free_dof_values(beta)

UnconstrainedFESpace()

In [6]:
FE_function_type() = Gridap.CellData.OperationCellField

FE_function_type (generic function with 1 method)

In [15]:
function EulerMEF(t_init,t_end,n_t,u_init)
    dt = (t_end - t_init)/n_t
    t = t_init
    u0 = interpolate_everywhere(u_init,U)
    u0 = get_free_dof_values(u0)
    
    a_convec(u,v) = ∫(betaf⋅∇(u)*v)*dΩ #conveccion
    a_masa(u,v)= ∫(v*u)*dΩ  #matriz masa
    
    uf = get_trial_fe_basis(V0)
    vf = get_fe_basis(V0)
    cell_dofs = get_cell_dof_ids(V0)
    assem = SparseMatrixAssembler(V0,V0)
    
    cell_mat_masa = a_masa(uf,vf)[Ω]
    data_masa = ([cell_mat_masa], [cell_dofs], [cell_dofs])
    M = assemble_matrix(assem,data_masa)
    
    cell_mat_convec = a_convec(uf,vf)[Ω]
    data_convec = ([cell_mat_convec], [cell_dofs], [cell_dofs])
    K = assemble_matrix(assem,data_convec)
    
    A = M + dt*K
    B = M*u0  # multiplicar matriz por el vector u0
    
    #TestPositividad(A,B)

    for i=1:n_t
        t += dt
        u_sol = A\B
        u0 = FEFunction(U,u_sol)
        
        B = M*u_sol  # multiplicar matriz por el vector u0
    end
    return u0
end 

EulerMEF (generic function with 1 method)

In [16]:
EulerMEF(t_init,t_end,N_intervalos,u_init)

SingleFieldFEFunction():
 num_cells: 780
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 13572624806936825544

In [17]:
t_n_f = 20;
t_n_c = 10;

F(t1, t0, u0) =  EulerMEF(t0, t1, t_n_f,u0)
G(t1, t0, u0) =  EulerMEF(t0, t1, t_n_c,u0)

G (generic function with 1 method)

In [18]:
dt = (t_end - t_init)/N_intervalos

u0_guardar = EulerMEF(t_init, t_init+dt, 1, u_init)

typeof(u0_guardar)
FE_function_type() = Gridap.CellData.OperationCellField

FE_function_type (generic function with 1 method)

In [19]:
function EDP_secuencial()
    U = Array{FE_function_type(),2}(undef, N_intervalos+1, n_iter+1);
    F_sol = Array{FE_function_type(),1}(undef, N_intervalos+1);


    # 1.a) Inicialización (aproximción grosera)
    U[1,1] = 1*u0_guardar
    

    for n=1:N_intervalos
        U[n+1,1] = 1*G( T[n+1],T[n],U[n,1] )
    end
    
        
    # 1.b) Inicialización etapas parareal
    for k=1:n_iter
        U[1,k+1] = 1*u0_guardar
    end

    # 2) Bucle parareal
    for k=1:n_iter
 
        # 2.a) Aproximación fina (paralela) en cada subintervalo
        for n = 1:N_intervalos
            F_sol[n] = 1*F( T[n+1], T[n], U[n,k] )
        end
        
        # 2.b) Corrección secuencial
        for n = 1:N_intervalos
            U[n+1, k+1] = F_sol[n] + G( T[n+1], T[n], U[n,k+1] ) - 
            G( T[n+1], T[n], U[n,k] )
        end
    end
    
    return U
    
end

EDP_secuencial (generic function with 1 method)

In [20]:
@time U1 = EDP_secuencial();

  8.580461 seconds (7.79 M allocations: 3.901 GiB, 3.52% gc time)


In [21]:
for i=1:N_intervalos+1
    sol = U1[i,end]
    writevtk(Ω,"Sol_numerica_sinFCT$(lpad(i,2,'0'))",cellfields=["sol" => sol])
end 


In [14]:
for i=1:N_intervalos+1
    int = ∫(U1[i,end])*dΩ
    println(Base.sum(int))
end 

0.0032628542315956527
0.0032628662234514953
0.003262891201366466
0.0032629867015266122
0.003263350552056406
0.003264439955521044
0.0032670495457075015
0.0032722579585825785
0.003281189684452923
0.003294629899737793
0.0033126259397286647
0.0033342480937772843
0.003357634159716497
0.00338032996831055
0.0033998213918608384
0.0034140881053620377
0.0034220171305499635
0.003423578546113748
0.003419750221102975
0.0034122483256995508
0.0034031559164891577
0.003394540982506958
0.0033881296364195674
0.0033850675782550327
0.003385778906521782
0.0033899218104194214
0.003396441269581396
0.003403719937425951
0.0034098221037226115
0.0034128104110626883
0.003411095038977807
0.0034037574421515735
0.0033907826801777034
0.003373141576037293
0.0033526887151833326
0.0033318815255208986
0.003313369861435065
0.0032995409223160457
0.0032921182624494278
0.0032919002810633245
0.0032986870549972917
0.0033113970940003264
0.003328332669934163
0.003347525258819906
0.003367085350757821
0.0033854903593040823
0.003401